In [15]:
import pandas as pd
import seaborn as sns
import numpy as np
import requests
import time, os
from bs4 import BeautifulSoup
import datetime
import re
from datetime import date
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

In [16]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = chromedriver

In [18]:
driver = webdriver.Chrome(chromedriver)

AttributeError: 'str' object has no attribute 'capabilities'